In [11]:
!git clone https://github.com/kumarvivek9088/aslsigndataset.git

fatal: destination path 'aslsigndataset' already exists and is not an empty directory.


In [12]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from PIL import Image
import os
import scipy
import scipy.ndimage



In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 48

train_generator = train_datagen.flow_from_directory(
    'D:/SLD/splitdataset48x48/train',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    'D:/SLD/splitdataset48x48/val',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 1819 images belonging to 7 classes.
Found 583 images belonging to 7 classes.


In [14]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['BLESS', 'HELLO', 'I_LOVE_YOU', 'NO', 'OKAY', 'YES', 'blank']


In [22]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 23, 23, 128)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 10, 10, 256)       0         
 g2D)                                                            
                                                                 
 dropout_19 (Dropout)        (None, 10, 10, 256)      

In [24]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )


In [25]:
!rm -rf Logs
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [26]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/50
37/37 [==============================] - 27s 668ms/step - loss: 1.9503 - accuracy: 0.1265 - val_loss: 1.9452 - val_accuracy: 0.1389
Epoch 2/50
37/37 [==============================] - 18s 484ms/step - loss: 1.9458 - accuracy: 0.1604 - val_loss: 1.9414 - val_accuracy: 0.1719
Epoch 3/50
37/37 [==============================] - 18s 484ms/step - loss: 1.8614 - accuracy: 0.1988 - val_loss: 1.7929 - val_accuracy: 0.2517
Epoch 4/50
37/37 [==============================] - 18s 478ms/step - loss: 1.7352 - accuracy: 0.2806 - val_loss: 1.7500 - val_accuracy: 0.2274
Epoch 5/50
37/37 [==============================] - 18s 486ms/step - loss: 1.6309 - accuracy: 0.2829 - val_loss: 1.7011 - val_accuracy: 0.2309
Epoch 6/50
37/37 [==============================] - 19s 502ms/step - loss: 1.5128 - accuracy: 0.3461 - val_loss: 1.4684 - val_accuracy: 0.3385
Epoch 7/50
37/37 [==============================] - 18s 492ms/step - loss: 1.3944 - accuracy: 0.3394 - val_loss: 1.2927 - val_accuracy: 0.4062

In [27]:
%load_ext tensorboard
%tensorboard --logdir Logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 15992), started 7 days, 13:10:08 ago. (Use '!kill 15992' to kill it.)

In [28]:
model_json = model.to_json()
with open("D:/SLD/signlanguagedetectionmodel48x48.json",'w') as json_file:
    json_file.write(model_json)
model.save("D:/SLD/signlanguagedetectionmodel48x48.h5")